<a href="https://colab.research.google.com/github/jakobzhao/geog458/blob/master/labs/lab02/youtube2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# created on April 14, 2021
# modified on Jan 2, 2022
# modified on April 20, 2023
# modified on January 5, 2024
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     A demo of collecting data from YouTube.

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.199/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,606 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,611 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/univers

In [ ]:
!pip uninstall chromedriver-autoinstaller -y

Found existing installation: chromedriver-autoinstaller 0.6.3
Uninstalling chromedriver-autoinstaller-0.6.3:
  Successfully uninstalled chromedriver-autoinstaller-0.6.3


In [ ]:
!google-chrome --version

/bin/bash: line 1: google-chrome: command not found


In [ ]:
!pip install chromedriver-autoinstaller
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()  # T

  Using cached chromedriver_autoinstaller-0.6.3-py3-none-any.whl (7.6 kB)


'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/120/chromedriver'

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

bot = webdriver.Chrome(options=chrome_options)


In [ ]:
from bs4 import BeautifulSoup
import time, datetime
import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from kora.selenium import wd as bot

# options = Options()
# options.add_argument("--headless")
# options.add_argument("--no-sandbox")
# bot = webdriver.Chrome("chromedriver", options=options)

In [ ]:
# The url where the data will be collected from.
url = "https://www.youtube.com/results?search_query=standing+rock"

# Input the targeting url to the bot, and the bot will load data from the url.
bot.get(url)

# An array to store all the video urls. If a video has been crawled, it would not be stored to the data frame.
video_urls = []
# An array to store the retrieved video details.
results = []


# variable i indicates the number of times that scrolls down a web page. In practice, you might want to develop different
# interaction approach to load and view the web pages.

for i in range(5):

    # Create a document object model (DOM) from the raw source of the crawled web page.
    # Since you are processing a html page, 'html.parser' is chosen.
    soup = BeautifulSoup(bot.page_source, 'html.parser')

    # Capture all the video items using find_all or findAll method.
    # To view the information of the html elements you want to collect, you need to inspect the raw source using Chrome Inspector.
    # To test whether you find the right html elements, you can use the pycharm debugger to examine the returned data.
    videos = soup.find_all('ytd-video-renderer', class_="style-scope ytd-item-section-renderer")[-20:] # 20 indicates only process the newly-acquired 20 entries.

    # iterate and process each video entry.
    for video in videos:

        # I prefer use the "try-except" statement to enable the program run without pausing due to unexpected errors.
        try:
            # video_url = video.find("a", class_="yt-simple-endpoint inline-block style-scope ytd-thumbnail").attrs["href"]
            # user_url = video.find("a", class_="yt-simple-endpoint style-scope yt-formatted-string").attrs["href"]
            # username = video.find("a", class_="yt-simple-endpoint style-scope yt-formatted-string").text
            # title = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
            # view_num = video.find_all("span", class_="style-scope ytd-video-meta-block")[0].text.replace(" views", "")
            # created_at = video.find_all("span", class_="style-scope ytd-video-meta-block")[0].text.replace(" ago", "")
            # shortdesc = video.find("yt-formatted-string", id="description-text").text
            # collected_at = datetime.datetime.now()

            video_url = video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer").attrs["href"]
            user_url = video.find("a", class_="yt-simple-endpoint style-scope yt-formatted-string").attrs["href"]
            username = video.find("a", class_="yt-simple-endpoint style-scope yt-formatted-string").text
            title = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
            metadata_items = video.find_all("span", class_="inline-metadata-item style-scope ytd-video-meta-block")
            view_num = metadata_items[0].text.replace(" views", "")
            created_at = metadata_items[1].text.replace(" ago", "")
            shortdesc = video.find("yt-formatted-string", class_="metadata-snippet-text style-scope ytd-video-renderer").text
            collected_at = datetime.datetime.now()

            # create a row in the dict format.
            row = {'video_url': video_url,
                    'user_url': user_url,
                    'username': username,
                    'title': title,
                    'view_num': view_num,
                    'created_at': created_at,
                    'shortdesc': shortdesc,
                    'collected_at': collected_at}

            # if a video has been added, this video would not be inserted to the results array,
            # otherwise, this video will be inserted.
            if video_url in video_urls:
                print("this video has already been added.")
            else:
                print(row)
                video_urls.append(video_url)
                results.append(row)
        except:
            pass

    # it is very important to enable the bot take some rest, and then resume to work.
    time.sleep(5)

    # Let the bot scrolls down to the bottom of the content element, most of the time the bot needs to scroll down to the bottom of the page.
    # like this statement: bot.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    bot.execute_script('window.scrollTo(0,  document.getElementById("content").scrollHeight);')

# terminate the bot object.
bot.close()

# Store the results as a pandas dataframe
df = pd.DataFrame(results)

# notify the completion of the crawling in the console.
print("the crawling task is finished.")

{'video_url': '/watch?v=Onyk7guvHK8&pp=ygUNc3RhbmRpbmcgcm9jaw%3D%3D', 'user_url': '/channel/UC5aqukzSeAj9etVpQpct88A', 'username': 'Taboo', 'title': 'Stand Up / Stand N Rock #NoDAPL (Official Video)', 'view_num': '4.4M', 'created_at': '7 years', 'shortdesc': "Hip Hop Caucus, creator of People's Climate Music, has partnered with Taboo, solo Hip Hop artist and member of the Grammy\xa0...", 'collected_at': datetime.datetime(2024, 1, 6, 4, 47, 18, 50422)}
{'video_url': '/watch?v=WFo-kIMtvCo&pp=ygUNc3RhbmRpbmcgcm9jaw%3D%3D', 'user_url': '/@seanfeucht', 'username': 'Sean Feucht', 'title': 'Standing on the Rock -  Sean Feucht - Times Square', 'view_num': '19K', 'created_at': '1 year', 'shortdesc': 'Follow Sean: https://www.facebook.com/sean.feucht https://twitter.com/seanfeucht https://www.instagram.com/seanfeucht/ Sign up\xa0...', 'collected_at': datetime.datetime(2024, 1, 6, 4, 47, 18, 54280)}
{'video_url': '/watch?v=2uYiHiJTN6Y&pp=ygUNc3RhbmRpbmcgcm9jaw%3D%3D', 'user_url': '/channel/UCr-Od

In [ ]:
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
#drive._mount('/gdrive')
drive.mount('/gdrive')

# the file path where to store the output csv on google drive
output_file = '/gdrive/My Drive/videos.csv'

# Save the dataframe as a csv file
df.to_csv(output_file, index=False)

Mounted at /gdrive


In [ ]:
# download the csv to your local computer
from google.colab import files
files.download(output_file)
print("the csv has been downloaded to your local computer. The program has been completed successfully.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
